In [4]:
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipelin

In [5]:
train_data = pd.read_csv('/Users/marlenawasiak/Desktop/Data_Collection/NSL_KDD_Train.csv')
test_data = pd.read_csv('/Users/marlenawasiak/Desktop/Data_Collection/NSL_KDD_Test.csv')

In [ ]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

common_columns = X_train.columns.intersection(X_test.columns)
X_train = X_train[common_columns]
X_test = X_test[common_columns]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from collections import Counter

# Step 1: Encode Categorical Features
categorical_columns = X_train.select_dtypes(include=['object']).columns

for col in categorical_columns:
    combined_categories = pd.concat([X_train[col], X_test[col]], axis=0).astype("category").cat.categories
    X_train[col] = pd.Categorical(X_train[col], categories=combined_categories)
    X_test[col] = pd.Categorical(X_test[col], categories=combined_categories)
    
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

clip_thresholds = {
    "0.1": X_train["0.1"].quantile(0.99),
    "0.2": X_train["0.2"].quantile(0.99)
}

X_train_clipped = X_train.copy()
X_test_clipped = X_test.copy()

for col, threshold in clip_thresholds.items():
    X_train_clipped[col] = X_train[col].clip(upper=threshold)
    X_test_clipped[col] = X_test[col].clip(upper=threshold)

scaler = QuantileTransformer(output_distribution='normal')
X_train_scaled = scaler.fit_transform(X_train_clipped)
X_test_scaled = scaler.transform(X_test_clipped)
X_train_scaled = np.clip(X_train_scaled, -3, 3)
X_test_scaled = np.clip(X_test_scaled, -3, 3)
all_labels = pd.concat([y_train, y_test], axis=0)
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print("Label classes:", label_encoder.classes_)

class_counts = Counter(y_train_encoded)
min_class_size = min(class_counts.values())
smote = SMOTE(random_state=42, k_neighbors=min(min_class_size - 1, 5))

X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train_encoded)

print(f"Original training data shape: {X_train_scaled.shape}, {y_train_encoded.shape}")
print(f"Resampled training data shape: {X_train_resampled.shape}, {y_train_resampled.shape}")
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

model = Sequential()
model.add(Dense(128, input_dim=X_train_resampled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(
    X_train_resampled, y_train_resampled, 
    epochs=40, batch_size=64, 
    validation_data=(X_test_scaled, y_test_encoded), 
    class_weight=class_weight_dict
)

y_pred_encoded = model.predict(X_test_scaled).argmax(axis=1)
test_accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print("Test Accuracy:", test_accuracy)
class_report = classification_report(
    y_test_encoded, 
    y_pred_encoded, 
    labels=range(len(label_encoder.classes_)), 
    target_names=label_encoder.classes_, 
    zero_division=1
)
print("Classification Report:\n", class_report)



Label classes: ['apache2' 'back' 'buffer_overflow' 'ftp_write' 'guess_passwd'
 'httptunnel' 'imap' 'ipsweep' 'land' 'loadmodule' 'mailbomb' 'mscan'
 'multihop' 'named' 'neptune' 'nmap' 'normal' 'perl' 'phf' 'pod'
 'portsweep' 'processtable' 'ps' 'rootkit' 'saint' 'satan' 'sendmail'
 'smurf' 'snmpgetattack' 'snmpguess' 'spy' 'sqlattack' 'teardrop'
 'udpstorm' 'warezclient' 'warezmaster' 'worm' 'xlock' 'xsnoop' 'xterm']
Original training data shape: (125972, 28), (125972,)
Resampled training data shape: (1548866, 28), (1548866,)


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 13s 502us/step - accuracy: 0.9488 - loss: 0.1647 - val_accuracy: 0.2907 - val_loss: 12.3759
Epoch 2/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 11s 457us/step - accuracy: 0.9814 - loss: 0.0574 - val_accuracy: 0.2906 - val_loss: 17.4702
Epoch 3/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 11s 453us/step - accuracy: 0.9840 - loss: 0.0493 - val_accuracy: 0.0920 - val_loss: 20.5927
Epoch 4/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 11s 454us/step - accuracy: 0.9851 - loss: 0.0463 - val_accuracy: 0.1941 - val_loss: 25.0936
Epoch 5/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 11s 455us/step - accuracy: 0.9860 - loss: 0.0434 - val_accuracy: 0.1413 - val_loss: 27.3747
Epoch 6/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 11s 457us/step - accuracy: 0.9859 - loss: 0.0435 - val_accuracy: 0.2487 - val_loss: 32.6592
Epoch 7/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 12s 497us/step - accuracy: 0.9865 - loss: 0.0419 - val_accuracy: 0.3689 - val_loss: 40.3727
Epoch 8/40
24202/24202 ━━━━━━━━━━━━━━━━━━━━ 11s 

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method
from art.estimators.classification import TensorFlowV2Classifier
import tensorflow as tf

classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=len(label_encoder.classes_),
    input_shape=(X_train_scaled.shape[1],),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
)

critical_features = [19, 23, 2, 26, 20] 
static_mask = np.zeros_like(X_test_scaled)
static_mask[:, critical_features] = 1  
def apply_static_mask(X_original, X_adv, mask):
    """
    Applies a static mask to perturb only selected features.
    Args:
        X_original: The original input data.
        X_adv: The adversarial examples.
        mask: A mask indicating which features to perturb (1 = perturb, 0 = keep original).
    Returns:
        Masked adversarial examples.
    """
    return X_original + (X_adv - X_original) * mask




In [ ]:

print("\n--- FGSM Attack with Mask ---")
for eps in [4.0]: 
    fgsm = FastGradientMethod(estimator=classifier, eps=eps)
    X_test_fgsm_adv = fgsm.generate(x=X_test_scaled)
    X_test_fgsm_adv = apply_static_mask(X_test_scaled, X_test_fgsm_adv, static_mask)
    X_test_fgsm_adv = np.clip(X_test_fgsm_adv, -3, 3)  
    y_pred_fgsm_encoded = model.predict(X_test_fgsm_adv).argmax(axis=1)
    fgsm_accuracy = accuracy_score(y_test_encoded, y_pred_fgsm_encoded)
    print(f"FGSM Adversarial Accuracy (eps={eps}):", fgsm_accuracy)
    print("FGSM Classification Report:\n", 
          classification_report(y_test_encoded, y_pred_fgsm_encoded,zero_division=1))


--- FGSM Attack with Static Mask ---
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step
FGSM Adversarial Accuracy (eps=4.0): 0.35106241405314287
FGSM Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00       737
           1       0.00      0.00      0.00       359
           2       1.00      0.00      0.00        20
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00      1231
           5       1.00      0.00      0.00       133
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00       141
           8       0.00      0.00      0.00         7
           9       1.00      0.00      0.00         2
          10       1.00      0.00      0.00       293
          11       1.00      0.00      0.00       996
          12       0.00      0.00      0.00        18
          13       1.00      0.00      0.00        17
          14       0.3

In [ ]:

print("\n--- PGD Attack with Static Mask ---")
pgd = ProjectedGradientDescent(
    estimator=classifier,
    norm=np.inf,
    eps=3.0,
    eps_step=0.02,
    max_iter=100,
    targeted=False
)
X_test_pgd_adv = pgd.generate(x=X_test_scaled)
X_test_pgd_adv = apply_static_mask(X_test_scaled, X_test_pgd_adv, static_mask)
X_test_pgd_adv = np.clip(X_test_pgd_adv, -3, 3)
y_pred_pgd_encoded = model.predict(X_test_pgd_adv).argmax(axis=1)
pgd_accuracy = accuracy_score(y_test_encoded, y_pred_pgd_encoded)
print("PGD Adversarial Accuracy (with masking):", pgd_accuracy)
print("PGD Classification Report:\n", 
      classification_report(y_test_encoded, y_pred_pgd_encoded,zero_division=1))


--- PGD Attack with Static Mask ---


PGD - Batches: 0it [00:00, ?it/s]

2024-11-23 19:08:50.637195: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step
PGD Adversarial Accuracy (with masking): 0.3210309186887282
PGD Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00       737
           1       1.00      0.00      0.00       359
           2       1.00      0.00      0.00        20
           3       1.00      0.00      0.00         3
           4       1.00      0.00      0.00      1231
           5       1.00      0.00      0.00       133
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00       141
           8       1.00      0.00      0.00         7
           9       1.00      0.00      0.00         2
          10       1.00      0.00      0.00       293
          11       1.00      0.00      0.00       996
          12       0.00      0.00      0.00        18
          13       1.00      0.00      0.00        17
          14       0.56      0.31      0.40      4656
    

In [ ]:
print("\n--- Carlini & Wagner Attack with Static Mask ---")
cw = CarliniL2Method(
    classifier=classifier,
    confidence=2.0,
    targeted=False,
    max_iter=100,
    learning_rate=0.01,
    binary_search_steps=5
)
X_test_cw_adv = cw.generate(x=X_test_scaled[:200])  
X_test_cw_adv = apply_static_mask(X_test_scaled[:200], X_test_cw_adv, static_mask[:200])
X_test_cw_adv = np.clip(X_test_cw_adv, -3, 3)
y_pred_cw_encoded = model.predict(X_test_cw_adv).argmax(axis=1)
cw_accuracy = accuracy_score(y_test_encoded[:200], y_pred_cw_encoded)
print("C&W Adversarial Accuracy (with masking):", cw_accuracy)
print("C&W Classification Report:\n", 
      classification_report(y_test_encoded[:200], y_pred_cw_encoded,zero_division=1))


--- Carlini & Wagner Attack with Static Mask ---


C&W L_2:   0%|          | 0/200 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 529us/step
C&W Adversarial Accuracy (with masking): 0.38
C&W Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00         3
           1       1.00      0.00      0.00         1
           2       1.00      0.00      0.00         1
           4       1.00      0.00      0.00        10
           5       1.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
          10       1.00      0.00      0.00         1
          11       1.00      0.00      0.00        10
          14       0.52      0.23      0.32        53
          15       1.00      0.00      0.00         2
          16       0.70      0.75      0.72        85
          19       1.00      0.00      0.00         2
          20       0.00      1.00      0.00         0
          21       1.00      0.00      0.00        10
          22       1.00      0.00      0.00         1
          24       1.0

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder, QuantileTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import numpy as np
from collections import Counter

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
X_train_scaled = quantile_transformer.fit_transform(X_train_scaled)
X_test_scaled = quantile_transformer.transform(X_test_scaled)
all_labels = np.concatenate([y_train, y_test])
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
class_counts = Counter(y_train_encoded)
min_class_size = min(class_counts.values())
smote = SMOTE(random_state=42, k_neighbors=min(min_class_size - 1, 5))

X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train_encoded)

print(f"Original training data shape: {X_train_scaled.shape}, {y_train_encoded.shape}")
print(f"Resampled training data shape: {X_train_resampled.shape}, {y_train_resampled.shape}")
num_classes = len(label_encoder.classes_) 
mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax') 
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = mlp_model.fit(
    X_train_resampled, y_train_resampled, 
    epochs=20, batch_size=32, 
    validation_data=(X_test_scaled, y_test_encoded)
)
y_pred_encoded = mlp_model.predict(X_test_scaled).argmax(axis=1)


Original training data shape: (125972, 28), (125972,)
Resampled training data shape: (1548866, 28), (1548866,)
Epoch 1/20


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


48403/48403 ━━━━━━━━━━━━━━━━━━━━ 22s 452us/step - accuracy: 0.9478 - loss: 0.1525 - val_accuracy: 0.2237 - val_loss: 16.0586
Epoch 2/20
48403/48403 ━━━━━━━━━━━━━━━━━━━━ 21s 424us/step - accuracy: 0.9806 - loss: 0.0608 - val_accuracy: 0.2605 - val_loss: 23.2003
Epoch 3/20
48403/48403 ━━━━━━━━━━━━━━━━━━━━ 21s 423us/step - accuracy: 0.9826 - loss: 0.0549 - val_accuracy: 0.1401 - val_loss: 34.5854
Epoch 4/20
48403/48403 ━━━━━━━━━━━━━━━━━━━━ 21s 426us/step - accuracy: 0.9847 - loss: 0.0495 - val_accuracy: 0.1093 - val_loss: 36.6616
Epoch 5/20
48403/48403 ━━━━━━━━━━━━━━━━━━━━ 21s 423us/step - accuracy: 0.9849 - loss: 0.0496 - val_accuracy: 0.2715 - val_loss: 44.8087
Epoch 6/20
48403/48403 ━━━━━━━━━━━━━━━━━━━━ 21s 428us/step - accuracy: 0.9855 - loss: 0.0481 - val_accuracy: 0.0940 - val_loss: 54.7231
Epoch 7/20
48403/48403 ━━━━━━━━━━━━━━━━━━━━ 21s 433us/step - accuracy: 0.9861 - loss: 0.0455 - val_accuracy: 0.1391 - val_loss: 67.1193
Epoch 8/20
48403/48403 ━━━━━━━━━━━━━━━━━━━━ 22s 450us/step 

In [ ]:

y_pred = mlp_model.predict(X_test_scaled).argmax(axis=1)
clean_accuracy = accuracy_score(y_test_encoded, y_pred)
clean_class_report = classification_report(y_test_encoded, y_pred, zero_division=1)
clean_conf_matrix = confusion_matrix(y_test_encoded, y_pred)
y_test_labels = label_encoder.inverse_transform(y_test_encoded)
y_pred_labels = label_encoder.inverse_transform(y_pred)
from sklearn.metrics import classification_report, confusion_matrix
decoded_class_report = classification_report(y_test_labels, y_pred_labels, zero_division=1)
decoded_conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
print("MLP Model Accuracy on Clean Data:", clean_accuracy)
print("Classification Report on Clean Data with Attack Names:\n", decoded_class_report)


705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step
MLP Model Accuracy on Clean Data: 0.14669742270327818
Classification Report on Clean Data with Attack Names:
                  precision    recall  f1-score   support

        apache2       1.00      0.00      0.00       737
           back       1.00      0.00      0.00       359
buffer_overflow       0.00      0.00      0.00        20
      ftp_write       1.00      0.00      0.00         3
   guess_passwd       1.00      0.00      0.00      1231
     httptunnel       1.00      0.00      0.00       133
           imap       1.00      0.00      0.00         1
        ipsweep       0.04      0.03      0.03       141
           land       1.00      0.00      0.00         7
     loadmodule       1.00      0.00      0.00         2
       mailbomb       1.00      0.00      0.00       293
          mscan       1.00      0.00      0.00       996
       multihop       1.00      0.00      0.00        18
          named       1.00      0.00      0.00  

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method
from art.estimators.classification import TensorFlowV2Classifier

classifier = TensorFlowV2Classifier(
    model=mlp_model,
    nb_classes=len(label_encoder.classes_),
    input_shape=(X_train_scaled.shape[1],),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
)

critical_features = [19, 23, 26, 2, 20, 27] 
def create_static_mask(data, critical_features):
    """
    Creates a mask with 1s for critical features and 0s elsewhere.
    Args:
        data: Dataset (numpy array or DataFrame).
        critical_features: List of critical feature indices.
    Returns:
        Mask of the same shape as data, with 1s for critical features.
    """
    mask = np.zeros_like(data)  
    mask[:, critical_features] = 1  
    return mask
static_mask = create_static_mask(X_test_scaled, critical_features)
def apply_static_mask(X_original, X_adv, mask):
    """
    Applies a static mask to perturb only selected features.
    Args:
        X_original: Original input data.
        X_adv: Adversarial examples.
        mask: Mask indicating which features to perturb (1 = perturb, 0 = keep original).
    Returns:
        Masked adversarial examples.
    """
    return X_original + (X_adv - X_original) * mask

In [ ]:

print("\n--- FGSM Attack with Dynamic Masking ---")
for eps in [3.0]:  
    fgsm = FastGradientMethod(estimator=classifier, eps=eps)
    X_test_fgsm_adv = fgsm.generate(x=X_test_scaled)
    X_test_fgsm_adv = apply_static_mask(X_test_scaled, X_test_fgsm_adv, static_mask)
    X_test_fgsm_adv = np.clip(X_test_fgsm_adv, -3, 3)  
    y_pred_fgsm_encoded = mlp_model.predict(X_test_fgsm_adv).argmax(axis=1)
    fgsm_accuracy = accuracy_score(y_test_encoded, y_pred_fgsm_encoded)
    print(f"FGSM Adversarial Accuracy (eps={eps}):", fgsm_accuracy)
    print("FGSM Classification Report:\n", 
          classification_report(y_test_encoded, y_pred_fgsm_encoded,zero_division=1))


--- FGSM Attack with Dynamic Masking ---
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step
FGSM Adversarial Accuracy (eps=3.0): 0.2854101051324136
FGSM Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00       737
           1       1.00      0.00      0.00       359
           2       0.00      0.00      0.00        20
           3       1.00      0.00      0.00         3
           4       1.00      0.00      0.00      1231
           5       1.00      0.00      0.00       133
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00       141
           8       1.00      0.00      0.00         7
           9       1.00      0.00      0.00         2
          10       1.00      0.00      0.00       293
          11       1.00      0.00      0.00       996
          12       0.00      0.00      0.00        18
          13       1.00      0.00      0.00        17
          14       

In [ ]:

print("\n--- PGD Attack with Dynamic Masking ---")
pgd = ProjectedGradientDescent(
    estimator=classifier,
    norm=np.inf,
    eps=3.0,
    eps_step=0.02,
    max_iter=100,
    targeted=False
)
X_test_pgd_adv = pgd.generate(x=X_test_scaled)
X_test_pgd_adv = apply_static_mask(X_test_scaled, X_test_pgd_adv, static_mask)
X_test_pgd_adv = np.clip(X_test_pgd_adv, -3, 3)
y_pred_pgd_encoded = mlp_model.predict(X_test_pgd_adv).argmax(axis=1)
pgd_accuracy = accuracy_score(y_test_encoded, y_pred_pgd_encoded)
print("PGD Adversarial Accuracy (with masking):", pgd_accuracy)
print("PGD Classification Report:\n", 
      classification_report(y_test_encoded, y_pred_pgd_encoded,zero_division=1))


--- PGD Attack with Dynamic Masking ---


PGD - Batches: 0it [00:00, ?it/s]

705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step
PGD Adversarial Accuracy (with masking): 0.18036641085924676
PGD Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00       737
           1       1.00      0.00      0.00       359
           2       0.00      0.00      0.00        20
           3       1.00      0.00      0.00         3
           4       1.00      0.00      0.00      1231
           5       1.00      0.00      0.00       133
           6       0.00      0.00      0.00         1
           7       0.04      0.01      0.02       141
           8       1.00      0.00      0.00         7
           9       1.00      0.00      0.00         2
          10       1.00      0.00      0.00       293
          11       1.00      0.00      0.00       996
          12       0.00      0.00      0.00        18
          13       1.00      0.00      0.00        17
          14       0.53      0.31      0.39      4656
   

In [ ]:

print("\n--- Carlini & Wagner Attack with Masking ---")
cw = CarliniL2Method(
    classifier=classifier,
    confidence=2.0,
    targeted=False,
    max_iter=100,
    learning_rate=0.01,
    binary_search_steps=5
)
X_test_cw_adv = cw.generate(x=X_test_scaled[:200]) 
X_test_cw_adv = apply_static_mask(X_test_scaled[:200], X_test_cw_adv, static_mask[:200])
X_test_cw_adv = np.clip(X_test_cw_adv, -3, 3)
y_pred_cw_encoded = mlp_model.predict(X_test_cw_adv).argmax(axis=1)
cw_accuracy = accuracy_score(y_test_encoded[:200], y_pred_cw_encoded)
print("C&W Adversarial Accuracy (with masking):", cw_accuracy)
print("C&W Classification Report:\n", 
      classification_report(y_test_encoded[:200], y_pred_cw_encoded,zero_division=1))


--- Carlini & Wagner Attack with Dynamic Masking ---


C&W L_2:   0%|          | 0/200 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step
C&W Adversarial Accuracy (with masking): 0.13
C&W Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00         3
           1       1.00      0.00      0.00         1
           2       1.00      0.00      0.00         1
           4       1.00      0.00      0.00        10
           5       1.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
          10       1.00      0.00      0.00         1
          11       1.00      0.00      0.00        10
          14       0.52      0.23      0.32        53
          15       0.00      0.00      0.00         2
          16       0.28      0.14      0.19        85
          19       1.00      0.00      0.00         2
          20       0.00      1.00      0.00         0
          21       1.00      0.00      0.00        10
          22       1.00      0.00      0.00         1
          23       0.0